In [1]:
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [5]:
# Get images
X = [] 
for filename in os.listdir('images/Train/'):
    X.append(img_to_array(load_img('images/Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X


#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.graph = tf.get_default_graph()

Instructions for updating:
Colocations handled automatically by placer.
225214464/225209952 [==============================] - 47s 0us/step


In [6]:
embed_input = Input(shape=(1000,))

#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)

In [7]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 10

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield ([X_batch, create_inception_embedding(grayscaled_rgb)], Y_batch)


#Train model      
model.compile(optimizer='rmsprop', loss='mse')
model.fit_generator(image_a_b_gen(batch_size), epochs=50, steps_per_epoch=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50


c:\users\yuki\miniconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


1/1 [==============================] - 51s 51s/step - loss: 0.0475
Epoch 2/50
1/1 [==============================] - 22s 22s/step - loss: 0.9703
Epoch 3/50
1/1 [==============================] - 19s 19s/step - loss: 0.9761
Epoch 4/50
1/1 [==============================] - 19s 19s/step - loss: 1.0216
Epoch 5/50
1/1 [==============================] - 20s 20s/step - loss: 1.0586
Epoch 6/50
1/1 [==============================] - 20s 20s/step - loss: 1.0002
Epoch 7/50
1/1 [==============================] - 20s 20s/step - loss: 0.9905
Epoch 8/50
1/1 [==============================] - 19s 19s/step - loss: 0.9724
Epoch 9/50
1/1 [==============================] - 19s 19s/step - loss: 1.0071
Epoch 10/50
1/1 [==============================] - 19s 19s/step - loss: 0.9906
Epoch 11/50
1/1 [==============================] - 19s 19s/step - loss: 1.0030
Epoch 12/50
1/1 [==============================] - 19s 19s/step - loss: 0.9475
Epoch 13/50
1/1 [==============================] - 19s 19s/step - loss: 

In [10]:
model.fit_generator(image_a_b_gen(batch_size), epochs=1, steps_per_epoch=50)

Epoch 1/1
50/50 [==============================] - 922s 18s/step - loss: 0.9764


In [13]:
color_me = []
for filename in os.listdir('images/Test/'):
    color_me.append(img_to_array(load_img('images/Test/'+filename)))
color_me = np.array(color_me, dtype=float)
gray_me = gray2rgb(rgb2gray(1.0/255*color_me))
color_me_embed = create_inception_embedding(gray_me)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))


# Test model
output = model.predict([color_me, color_me_embed])
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

c:\users\yuki\miniconda3\lib\site-packages\skimage\color\colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 56695 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
c:\users\yuki\miniconda3\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))
c:\users\yuki\miniconda3\lib\site-packages\skimage\color\colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 51077 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
c:\users\yuki\miniconda3\lib\site-packages\skimage\color\colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 64004 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invalid[0].size)
c:\users\yuki\miniconda3\lib\site-packages\skimage\color\colorconv.py:993: UserWarning: Color data out of range: Z < 0 in 13712 pixels
  warn('Color data out of range: Z < 0 in %s pixels' % invali